In [1]:
# Encoder 

import torch
import torch.nn as nn
import torch.nn.functional as F
from src.layers import ResnetBlockFC
from torch_scatter import scatter_mean, scatter_max
from src.common import coordinate2index, normalize_coordinate, normalize_3d_coordinate, map2local
from src.encoder.unet import UNet
from src.encoder.unet3d import UNet3D


class LocalPoolPointnet(nn.Module):
    ''' PointNet-based encoder network with ResNet blocks for each point.
        Number of input points are fixed.
    
    Args:
        c_dim (int): dimension of latent code c
        dim (int): input points dimension
        hidden_dim (int): hidden dimension of the network
        scatter_type (str): feature aggregation when doing local pooling
        unet (bool): weather to use U-Net
        unet_kwargs (str): U-Net parameters
        unet3d (bool): weather to use 3D U-Net
        unet3d_kwargs (str): 3D U-Net parameters
        plane_resolution (int): defined resolution for plane feature
        grid_resolution (int): defined resolution for grid feature 
        plane_type (str): feature type, 'xz' - 1-plane, ['xz', 'xy', 'yz'] - 3-plane, ['grid'] - 3D grid volume
        padding (float): conventional padding paramter of ONet for unit cube, so [-0.5, 0.5] -> [-0.55, 0.55]
        n_blocks (int): number of blocks ResNetBlockFC layers
    '''

    def __init__(self, c_dim=128, dim=3, hidden_dim=128, scatter_type='max', 
                 unet=False, unet_kwargs=None, unet3d=False, unet3d_kwargs=None, 
                 plane_resolution=None, grid_resolution=None, plane_type='xz', padding=0.1, n_blocks=5):
        super().__init__()
        self.c_dim = c_dim

        self.fc_pos = nn.Linear(dim, 2*hidden_dim)
        self.blocks = nn.ModuleList([
            ResnetBlockFC(2*hidden_dim, hidden_dim) for i in range(n_blocks)
        ])
        self.fc_c = nn.Linear(hidden_dim, c_dim)

        self.actvn = nn.ReLU()
        self.hidden_dim = hidden_dim

        if unet:
            self.unet = UNet(c_dim, in_channels=c_dim, **unet_kwargs)
        else:
            self.unet = None

        if unet3d:
            self.unet3d = UNet3D(**unet3d_kwargs)
        else:
            self.unet3d = None

        self.reso_plane = plane_resolution
        self.reso_grid = grid_resolution
        self.plane_type = plane_type
        self.padding = padding

        if scatter_type == 'max':
            self.scatter = scatter_max
        elif scatter_type == 'mean':
            self.scatter = scatter_mean
        else:
            raise ValueError('incorrect scatter type')


    def generate_plane_features(self, p, c, plane='xz'):
        # acquire indices of features in plane
        xy = normalize_coordinate(p.clone(), plane=plane, padding=self.padding) # normalize to the range of (0, 1)
        index = coordinate2index(xy, self.reso_plane)

        # scatter plane features from points
        fea_plane = c.new_zeros(p.size(0), self.c_dim, self.reso_plane**2)
        c = c.permute(0, 2, 1) # B x 512 x T
        fea_plane = scatter_mean(c, index, out=fea_plane) # B x 512 x reso^2
        fea_plane = fea_plane.reshape(p.size(0), self.c_dim, self.reso_plane, self.reso_plane) # sparce matrix (B x 512 x reso x reso)

        # process the plane features with UNet
        if self.unet is not None:
            fea_plane = self.unet(fea_plane)

        return fea_plane

    def generate_grid_features(self, p, c):
        p_nor = normalize_3d_coordinate(p.clone(), padding=self.padding)
        index = coordinate2index(p_nor, self.reso_grid, coord_type='3d')
        # scatter grid features from points
        fea_grid = c.new_zeros(p.size(0), self.c_dim, self.reso_grid**3)
        c = c.permute(0, 2, 1)
        fea_grid = scatter_mean(c, index, out=fea_grid) # B x C x reso^3
        fea_grid = fea_grid.reshape(p.size(0), self.c_dim, self.reso_grid, self.reso_grid, self.reso_grid) # sparce matrix (B x 512 x reso x reso)

        if self.unet3d is not None:
            fea_grid = self.unet3d(fea_grid)

        return fea_grid

    def pool_local(self, xy, index, c):
        bs, fea_dim = c.size(0), c.size(2)
        keys = xy.keys()

        c_out = 0
        for key in keys:
            # scatter plane features from points
            if key == 'grid':
                fea = self.scatter(c.permute(0, 2, 1), index[key], dim_size=self.reso_grid**3)
            else:
                fea = self.scatter(c.permute(0, 2, 1), index[key], dim_size=self.reso_plane**2)
            if self.scatter == scatter_max:
                fea = fea[0]
            # gather feature back to points
            fea = fea.gather(dim=2, index=index[key].expand(-1, fea_dim, -1))
            c_out += fea
        return c_out.permute(0, 2, 1)


    def forward(self, p):
        batch_size, T, D = p.size()

        # acquire the index for each point
        coord = {}
        index = {}
        if 'xz' in self.plane_type:
            coord['xz'] = normalize_coordinate(p.clone(), plane='xz', padding=self.padding)
            index['xz'] = coordinate2index(coord['xz'], self.reso_plane)
        if 'xy' in self.plane_type:
            coord['xy'] = normalize_coordinate(p.clone(), plane='xy', padding=self.padding)
            index['xy'] = coordinate2index(coord['xy'], self.reso_plane)
        if 'yz' in self.plane_type:
            coord['yz'] = normalize_coordinate(p.clone(), plane='yz', padding=self.padding)
            index['yz'] = coordinate2index(coord['yz'], self.reso_plane)
        if 'grid' in self.plane_type:
            coord['grid'] = normalize_3d_coordinate(p.clone(), padding=self.padding)
            index['grid'] = coordinate2index(coord['grid'], self.reso_grid, coord_type='3d')
        
        net = self.fc_pos(p)

        net = self.blocks[0](net)
        for block in self.blocks[1:]:
            pooled = self.pool_local(coord, index, net)
            net = torch.cat([net, pooled], dim=2)
            net = block(net)

        c = self.fc_c(net)

        fea = {}
        if 'grid' in self.plane_type:
            fea['grid'] = self.generate_grid_features(p, c)
        if 'xz' in self.plane_type:
            fea['xz'] = self.generate_plane_features(p, c, plane='xz')
        if 'xy' in self.plane_type:
            fea['xy'] = self.generate_plane_features(p, c, plane='xy')
        if 'yz' in self.plane_type:
            fea['yz'] = self.generate_plane_features(p, c, plane='yz')

        return fea


In [2]:
# Decoder

import torch
import torch.nn as nn
import torch.nn.functional as F
from src.layers import ResnetBlockFC
from src.common import normalize_coordinate, normalize_3d_coordinate, map2local


class LocalDecoder(nn.Module):
    ''' Decoder.
        Instead of conditioning on global features, on plane/volume local features.

    Args:
        dim (int): input dimension
        c_dim (int): dimension of latent conditioned code c
        hidden_size (int): hidden size of Decoder network
        n_blocks (int): number of blocks ResNetBlockFC layers
        leaky (bool): whether to use leaky ReLUs
        sample_mode (str): sampling feature strategy, bilinear|nearest
        padding (float): conventional padding paramter of ONet for unit cube, so [-0.5, 0.5] -> [-0.55, 0.55]
    '''

    def __init__(self, dim=3, c_dim=128,
                 hidden_size=256, n_blocks=5, leaky=False, sample_mode='bilinear', padding=0.1):
        super().__init__()
        self.c_dim = c_dim
        self.n_blocks = n_blocks

        if c_dim != 0:
            self.fc_c = nn.ModuleList([
                nn.Linear(c_dim, hidden_size) for i in range(n_blocks)
            ])


        self.fc_p = nn.Linear(dim, hidden_size)

        self.blocks = nn.ModuleList([
            ResnetBlockFC(hidden_size) for i in range(n_blocks)
        ])

        self.fc_out = nn.Linear(hidden_size, 1)

        if not leaky:
            self.actvn = F.relu
        else:
            self.actvn = lambda x: F.leaky_relu(x, 0.2)

        self.sample_mode = sample_mode
        self.padding = padding
    

    def sample_plane_feature(self, p, c, plane='xz'):
        xy = normalize_coordinate(p.clone(), plane=plane, padding=self.padding) # normalize to the range of (0, 1)
        xy = xy[:, :, None].float()
        vgrid = 2.0 * xy - 1.0 # normalize to (-1, 1)
        c = F.grid_sample(c, vgrid, padding_mode='border', align_corners=True, mode=self.sample_mode).squeeze(-1)
        return c

    def sample_grid_feature(self, p, c):
        p_nor = normalize_3d_coordinate(p.clone(), padding=self.padding) # normalize to the range of (0, 1)
        p_nor = p_nor[:, :, None, None].float()
#         print(np.unique(p.clone().cpu().numpy()),np.unique(p_nor.cpu().numpy()))
        vgrid = 2.0 * p_nor - 1.0 # normalize to (-1, 1)
        # acutally trilinear interpolation if mode = 'bilinear'
        c = F.grid_sample(c, vgrid, padding_mode='border', align_corners=True, mode=self.sample_mode).squeeze(-1).squeeze(-1)
        return c


    def forward(self, p, c_plane, **kwargs):
        if self.c_dim != 0:
            plane_type = list(c_plane.keys())
            c = 0
            if 'grid' in plane_type:
                c += self.sample_grid_feature(p, c_plane['grid'])
            if 'xz' in plane_type:
                c += self.sample_plane_feature(p, c_plane['xz'], plane='xz')
            if 'xy' in plane_type:
                c += self.sample_plane_feature(p, c_plane['xy'], plane='xy')
            if 'yz' in plane_type:
                c += self.sample_plane_feature(p, c_plane['yz'], plane='yz')
            c = c.transpose(1, 2)

        p = p.float()
        net = self.fc_p(p)

        for i in range(self.n_blocks):
            if self.c_dim != 0:
                net = net + self.fc_c[i](c)

            net = self.blocks[i](net)

        out = self.fc_out(self.actvn(net))
        out = out.squeeze(-1)

        return out


In [3]:
# CON

import torch
import torch.nn as nn
from torch import distributions as dist
# from src.conv_onet.models import decoder


class ConvolutionalOccupancyNetwork(nn.Module):
    ''' Occupancy Network class.

    Args:
        decoder (nn.Module): decoder network
        encoder (nn.Module): encoder network
        device (device): torch device
    '''

    def __init__(self, decoder, encoder=None, device=None):
        super().__init__()
        
        self.decoder = decoder.to(device)

        if encoder is not None:
            self.encoder = encoder.to(device)
        else:
            self.encoder = None

        self._device = device

    def forward(self, p, inputs, sample=True, **kwargs):
        ''' Performs a forward pass through the network.

        Args:
            p (tensor): sampled points
            inputs (tensor): conditioning input
            sample (bool): whether to sample for z
        '''
        #############
        if isinstance(p, dict):
            batch_size = p['p'].size(0)
        else:
            batch_size = p.size(0)
        c = self.encode_inputs(inputs)
        p_r = self.decode(p, c, **kwargs)
        return p_r

    def encode_inputs(self, inputs):
        ''' Encodes the input.

        Args:
            input (tensor): the input
        '''

        if self.encoder is not None:
            c = self.encoder(inputs)
        else:
            # Return inputs?
            c = torch.empty(inputs.size(0), 0)

        return c

    def decode(self, p, c, **kwargs):
        ''' Returns occupancy probabilities for the sampled points.

        Args:
            p (tensor): points
            c (tensor): latent conditioned code c
        '''

        logits = self.decoder(p, c, **kwargs)
        p_r = dist.Bernoulli(logits=logits)
        return p_r

    def to(self, device):
        ''' Puts the model to the device.

        Args:
            device (device): pytorch device
        '''
        model = super().to(device)
        model._device = device
        return model


In [4]:
class SubsamplePointcloud(object):
    ''' Point cloud subsampling transformation class.

    It subsamples the point cloud data.

    Args:
        N (int): number of points to be subsampled
    '''
    def __init__(self, N):
        self.N = N

    def __call__(self, data):
        ''' Calls the transformation.

        Args:
            data (dict): data dictionary
        '''
        data_out = data.copy()
        points = data[None]
        normals = data['normals']

        indices = np.random.randint(points.shape[0], size=self.N)
        data_out[None] = points[indices, :]
        data_out['normals'] = normals[indices, :]

        return data_out
    
    
class SubsamplePoints(object):
    ''' Points subsampling transformation class.

    It subsamples the points data.

    Args:
        N (int): number of points to be subsampled
    '''
    def __init__(self, N):
        self.N = N

    def __call__(self, data):
        ''' Calls the transformation.

        Args:
            data (dictionary): data dictionary
        '''

        points = data[None]
        occ = data['occ']

        data_out = data.copy()
        if isinstance(self.N, int):
            idx = np.random.randint(points.shape[0], size=self.N)
            data_out.update({
                None: points[idx, :],
                'occ':  occ[idx],
            })
#         else:
#             Nt_out, Nt_in = self.N
#             occ_binary = (occ >= 0.5)
#             points0 = points[~occ_binary]
#             points1 = points[occ_binary]

#             idx0 = np.random.randint(points0.shape[0], size=Nt_out)
#             idx1 = np.random.randint(points1.shape[0], size=Nt_in)

#             points0 = points0[idx0, :]
#             points1 = points1[idx1, :]
#             points = np.concatenate([points0, points1], axis=0)

#             occ0 = np.zeros(Nt_out, dtype=np.float32)
#             occ1 = np.ones(Nt_in, dtype=np.float32)
#             occ = np.concatenate([occ0, occ1], axis=0)

#             volume = occ_binary.sum() / len(occ_binary)
#             volume = volume.astype(np.float32)

#             data_out.update({
#                 None: points,
#                 'occ': occ,
#                 'volume': volume,
#             })
        return data_out

In [5]:
# Carla Dataloader 

# import os
# import numpy as np
# import random
# import json

# import torch
# import torch.nn.functional as F
# from torch.utils.data import Dataset
# from torch.utils import data

# class CarlaDataset(Dataset):
#     """Carla Simulation Dataset for 3D mapping project
    
#     Access to the processed data, including evaluation labels predictions velodyne poses times
#     """
#     def __init__(self, directory,
#         device='cuda',
#         num_frames=1,
#         voxelize_input=False,
#         binary_counts=True,
#         random_flips=False,       
#         ):
#         '''Constructor.
#         Parameters:
#             directory: directory to the dataset
#         '''

#         self.voxelize_input = voxelize_input
#         self.binary_counts = binary_counts
#         self._directory = directory
#         self._num_frames = num_frames
#         self.device = device
#         self.random_flips = random_flips
        
#         self._scenes = sorted(os.listdir(self._directory))
#         self._scenes = [os.path.join(scene, "cartesian") for scene in self._scenes]

#         self._num_scenes = len(self._scenes)
#         self._num_frames_scene = []

#         self._velodyne_list = []
#         self._eval_labels = []
#         self._frames_list = []

#         for scene in self._scenes:
#             velodyne_dir = os.path.join(self._directory, scene, 'pointcloud')
#             eval_dir = os.path.join(self._directory, scene, 'points')
#             self._num_frames_scene.append(len(os.listdir(velodyne_dir)))
            
#             frames_list = [os.path.splitext(filename)[0] for filename in sorted(os.listdir(velodyne_dir))]
#             self._frames_list.extend(frames_list)

#             self._velodyne_list.extend([os.path.join(velodyne_dir, 'pointcloud_' + str(frame).zfill(6) +'.npz') for frame in range(len(frames_list))])
#             self._eval_labels.extend([os.path.join(eval_dir, 'points_' + str(frame).zfill(6) +'.npz') for frame in range(len(frames_list))])

#         self._cum_num_frames = np.cumsum(np.array(self._num_frames_scene) - self._num_frames + 1)
#     # Use all frames, if there is no data then zero pad
#     def __len__(self):
#         return sum(self._num_frames_scene)
    
#     def collate_fn(self, batch):
#         #input_batch = [bi[0] for bi in data]
# #         output_batch = [bi[1] for bi in data]
#         #return input_batch
        
#         batch = list(filter(lambda x: x is not None, batch))
#         return data.dataloader.default_collate(batch)

#     def get_file_path(self, idx):
#         print(self._frames_list[idx])

        
#     def __getitem__(self, idx):

#         pointcloud = np.load(self._velodyne_list[idx])
#         points = np.load(self._eval_labels[idx])
        
#         pc_field = {}
#         pc_field[None] = pointcloud['points']
#         pc_field['normals'] = pointcloud['normals']
        
#         pc_field_sub = SubsamplePointcloud(10000)
#         pc_field = pc_field_sub(pc_field)
        
#         points_field = {}
#         points_field[None] = points['points']
#         points_field['occ'] = points['occupancies']
        
#         points_field_sub = SubsamplePoints(2048)
#         points_field = points_field_sub(points_field)
        
# #         print(pc_field[None].shape, pc_field['normals'].shape)
# #         print(points_field[None].shape, points_field['occupancies'].shape)
        
#         fields = {}
#         data = {}
        
#         #for pc_field,points_field in carla_ds:

#         fields['points'] = points_field
        
#         if pc_field is not None:
#             fields['inputs'] = pc_field

#         for field_name, field in fields.items():
#             field_data = field
#             for k, v in field_data.items():
#                 if k is None:
#                     data[field_name] = v
#                 else:
#                     data['%s.%s' % (field_name, k)] = v
# #         print(data['points'].shape, data['points.occ'].shape, data['inputs'].shape)
#         return data

In [6]:
def unpack(compressed):
    ''' given a bit encoded voxel grid, make a normal voxel grid out of it.  '''
    uncompressed = np.zeros(compressed.shape[0] * 8, dtype=np.uint8)
    uncompressed[::8] = compressed[:] >> 7 & 1
    uncompressed[1::8] = compressed[:] >> 6 & 1
    uncompressed[2::8] = compressed[:] >> 5 & 1
    uncompressed[3::8] = compressed[:] >> 4 & 1
    uncompressed[4::8] = compressed[:] >> 3 & 1
    uncompressed[5::8] = compressed[:] >> 2 & 1
    uncompressed[6::8] = compressed[:] >> 1 & 1
    uncompressed[7::8] = compressed[:] & 1

    return uncompressed

In [7]:
# rellis3d dataset
import os
import numpy as np
import random
import json
import copy

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils import data

class RellisDataset(Dataset):
    """Rellis3D Dataset for convOccn
    
    Access to the processed data, including evaluation labels predictions velodyne poses times
    """
    def __init__(self, directory,
        device='cuda',
        num_frames=1,
        voxelize_input=False,
        binary_counts=True,
        random_flips=False,   
        model_setting = 'train'
        ):
        '''Constructor.
        Parameters:
            directory: directory to the dataset
        '''

        self.voxelize_input = voxelize_input
        self.binary_counts = binary_counts
        self._directory = directory
        self._num_frames = num_frames
        self.device = device
        self.random_flips = random_flips
        
        self._scenes = sorted(os.listdir(self._directory))
#         self._scenes = [os.path.join(scene, "cartesian") for scene in self._scenes]

        self._num_scenes = len(self._scenes)
        #self._num_frames_scene = []
        self._num_frames_scene = 0
        
        self._velodyne_list = []
        self._eval_labels = []
        self._frames_list = []
        
        split_dir = os.path.join(self._directory, "pt_"+model_setting+".lst")

        # Generate list of scenes and indices to iterate over
        self._scenes_list = []
        self._index_list = []

        with open(split_dir, 'r') as split_file:
            for line in split_file:
                image_path = line.split(' ')
                image_path_lst = image_path[0].split('/')
                scene_num = image_path_lst[0]
                frame_index = int(image_path_lst[2][0:6])
                self._scenes_list.append(scene_num)
                self._index_list.append(frame_index)

        for i in range(len(self._scenes_list)):
            velodyne_dir = os.path.join(self._directory, self._scenes_list[i], 'velodyne')
            eval_dir = os.path.join(self._directory, self._scenes_list[i], 'voxels')
            self._num_frames_scene += 1
            
            self._velodyne_list.extend([os.path.join(velodyne_dir, str(self._index_list[i]).zfill(6) + '.bin')])
            self._eval_labels.extend([os.path.join(eval_dir, str(self._index_list[i]).zfill(6) + '.label')])
        
#             velodyne_dir = os.path.join(self._directory, scene, 'velodyne')
#             eval_dir = os.path.join(self._directory, scene, 'voxels')
#             self._num_frames_scene.append(len(os.listdir(velodyne_dir)))
            
#             frames_list = [os.path.splitext(filename)[0] for filename in sorted(os.listdir(velodyne_dir))]
#             self._frames_list.extend(frames_list)

#             self._velodyne_list.extend([os.path.join(velodyne_dir, str(frame).zfill(6) +'.bin') for frame in range(len(frames_list))])
#             self._eval_labels.extend([os.path.join(eval_dir, str(frame).zfill(6) +'.label') for frame in range(len(frames_list))])

#         self._cum_num_frames = np.cumsum(np.array(self._num_frames_scene) - self._num_frames + 1)

    # Use all frames, if there is no data then zero pad
    def __len__(self):
#         return sum(self._num_frames_scene)
        return self._num_frames_scene
    
    def collate_fn(self, batch):
        batch = list(filter(lambda x: x is not None, batch))
        return data.dataloader.default_collate(batch)

    
    def __getitem__(self, idx):
        
        pointcloud = np.fromfile(self._velodyne_list[idx],dtype=np.float32).reshape(-1,4)[:,:3]

        
        points_occ = np.fromfile(self._eval_labels[idx], dtype=np.uint8)
        
        mask = points_occ != 0
        points_occ = points_occ[mask]
#         print(np.unique(points_occ, return_counts=True))
        
        # points
        min_bound = [-25.6, -25.6, -2]
        max_bound = [25.6, 25.6, 1.2]
        grid_size = [256, 256, 16]
        coor_ranges = min_bound + max_bound
        voxel_sizes = [abs(coor_ranges[3] - coor_ranges[0]) / grid_size[0], 
                    abs(coor_ranges[4] - coor_ranges[1]) / grid_size[1],
                    abs(coor_ranges[5] - coor_ranges[2]) / grid_size[2]]

        x = np.linspace(min_bound[0], max_bound[0], num=int(grid_size[0])) + voxel_sizes[0] / 2
        y = np.linspace(min_bound[1], max_bound[1], num=int(grid_size[1])) + voxel_sizes[1] / 2
        z = np.linspace(min_bound[2], max_bound[2], num=int(grid_size[2])) + voxel_sizes[2] / 2
        xv, yv, zv = np.meshgrid(x, y, z, indexing="ij")

        points = np.stack((xv, yv, zv))
        points = np.moveaxis(points,0,-1).reshape(-1,3)
        points = points[mask]
        
        # Filter points outside of voxel grid
        grid_point_mask= np.all(
            (pointcloud < max_bound) & (pointcloud >= min_bound), axis=1)
        pointcloud = pointcloud[grid_point_mask, :]
        
        # normalize
        pointcloud /= (2*np.max(pointcloud))
        points /= (2*np.max(points))
#         print('nm:',np.unique(pointcloud, return_counts=True))
#         print('p:',np.unique(pointcloud, return_counts=True))
        
#         pocc_t = copy.deepcopy(points_occ)
        
#         points_occ[pocc_t != 40] = 1
#         points_occ[pocc_t == 40] = 0
        points_occ[points_occ == 40] = 0
        points_occ[points_occ != 0 ] = 1
                
        pc_field = {}
        pc_field[None] = pointcloud
        pc_field['normals'] = np.zeros((pointcloud.shape[0],1))
    
        pc_field_sub = SubsamplePointcloud(10000) #10000, 131072
        pc_field = pc_field_sub(pc_field)
        

        # normalize points
        
        points_field = {}
        points_field[None] = points
        points_field['occ'] = points_occ
        
        points_field_sub = SubsamplePoints(2048) #2048, 131072
        points_field = points_field_sub(points_field)
        
    #         print(pc_field[None].shape, pc_field['normals'].shape)
    #         print(points_field[None].shape, points_field['occupancies'].shape)
        
        fields = {}
        data = {}
        
        #for pc_field,points_field in carla_ds:

        fields['points'] = points_field
        
        if pc_field is not None:
            fields['inputs'] = pc_field

        for field_name, field in fields.items():
            field_data = field
            for k, v in field_data.items():
                if k is None:
                    data[field_name] = v
                else:
                    data['%s.%s' % (field_name, k)] = v
#         print(data['points'].shape, data['points.occ'].shape, data['inputs'].shape)
        return data

In [8]:
# Dataloader
from torch.utils.data import DataLoader


r3d_dir = "/home/jason/rellis3d"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
carla_ds_train = RellisDataset(directory=r3d_dir, device=device, model_setting='train')
carla_ds_eval = RellisDataset(directory=r3d_dir, device=device, model_setting='val')

dataloader_train = DataLoader(carla_ds_train, batch_size=4, shuffle=True, collate_fn=carla_ds_train.collate_fn, num_workers=4)
dataloader_eval = DataLoader(carla_ds_eval, batch_size=1, shuffle=False, collate_fn=carla_ds_eval.collate_fn, num_workers=4)
# o1= next(iter(Dataloader))
#print(list(o1[0].keys()), list(o2[0].keys()))
# test = o2[0]
# nz = np.sum(test!=0)
# print(o2[0].shape, nz)


cuda


In [9]:
# Model (configs/pointcloud/room_grid32.yaml)
# src.conv_onet.config.py get_model

encoder_kwargs = {'hidden_dim': 32,
                  'plane_type': 'grid',
                  'grid_resolution': 64, #32
                  'unet3d': True,
                  'unet3d_kwargs': {
                    'num_levels': 4,#3
                    'f_maps': 32,
                    'in_channels': 32,
                    'out_channels': 32}}

decoder_kwargs = {
    'sample_mode': 'bilinear', # bilinear / nearest
    'hidden_size': 32 }

c_dim = 32
dim = 3
padding = 0.1
decoder = LocalDecoder(dim=dim, c_dim=c_dim, padding=padding,
        **decoder_kwargs)

encoder = LocalPoolPointnet(dim=dim, c_dim=c_dim, padding=padding,
            **encoder_kwargs)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ConvolutionalOccupancyNetwork(
        decoder, encoder, device=device
    )

print(model)



ConvolutionalOccupancyNetwork(
  (decoder): LocalDecoder(
    (fc_c): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Linear(in_features=32, out_features=32, bias=True)
      (4): Linear(in_features=32, out_features=32, bias=True)
    )
    (fc_p): Linear(in_features=3, out_features=32, bias=True)
    (blocks): ModuleList(
      (0): ResnetBlockFC(
        (fc_0): Linear(in_features=32, out_features=32, bias=True)
        (fc_1): Linear(in_features=32, out_features=32, bias=True)
        (actvn): ReLU()
      )
      (1): ResnetBlockFC(
        (fc_0): Linear(in_features=32, out_features=32, bias=True)
        (fc_1): Linear(in_features=32, out_features=32, bias=True)
        (actvn): ReLU()
      )
      (2): ResnetBlockFC(
        (fc_0): Linear(in_features=32, out_features=32, bias=True)
        (fc_1): Linear(in_features=32

In [10]:
# compute loss 

def compute_loss(data):
    ''' Computes the loss.

    Args:
        data (dict): data dictionary
    '''    
    p = data.get('points').to(device)
    occ = data.get('points.occ').to(device)
    inputs = data.get('inputs', torch.empty(p.size(0), 0)).to(device)

    if 'pointcloud_crop' in data.keys():
        # add pre-computed index
        inputs = add_key(inputs, data.get('inputs.ind'), 'points', 'index', device=device)
        inputs['mask'] = data.get('inputs.mask').to(device)
        # add pre-computed normalized coordinates
        p = add_key(p, data.get('points.normalized'), 'p', 'p_n', device=device)

    c = model.encode_inputs(inputs)

    kwargs = {}
    # General points
    logits = model.decode(p, c, **kwargs).logits
    loss_i = F.binary_cross_entropy_with_logits(
        logits, occ.float(), reduction='none')
    loss = loss_i.sum(-1).mean()
    return loss

In [11]:
# compute iou
def compute_iou(occ1, occ2):
    ''' Computes the Intersection over Union (IoU) value for two sets of
    occupancy values.

    Args:
        occ1 (tensor): first set of occupancy values
        occ2 (tensor): second set of occupancy values
    '''
    occ1 = np.asarray(occ1)
    occ2 = np.asarray(occ2)

    # Put all data in second dimension
    # Also works for 1-dimensional data
    if occ1.ndim >= 2:
        occ1 = occ1.reshape(occ1.shape[0], -1)
    if occ2.ndim >= 2:
        occ2 = occ2.reshape(occ2.shape[0], -1)

    # Convert to boolean values
    occ1 = (occ1 >= 0.5)
    occ2 = (occ2 >= 0.5)

    # Compute IOU
    area_union = (occ1 | occ2).astype(np.float32).sum(axis=-1)
    area_intersect = (occ1 & occ2).astype(np.float32).sum(axis=-1)
    
    print(area_union, area_intersect)

    iou = (area_intersect / area_union)

    return iou

def eval_step(data):
    model.eval()
    

In [12]:

# Intersection, union for one frame
def iou_one_frame(pred, target, n_classes=23):
    pred = np.asarray(pred)
    target = np.asarray(target)
    pred = pred.reshape(-1)
    target = target.reshape(-1)
    intersection = np.zeros(n_classes)
    union = np.zeros(n_classes)

    for cls in range(n_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection[cls] = (pred_inds[target_inds]).sum() # Cast to long to prevent overflows
        union[cls] = pred_inds.sum() + target_inds.sum() - intersection[cls]
    return intersection, union

In [13]:
# test
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import shutil

save_dir = '/home/jason/convolutional_occupancy_networks/out/r3d/'
write_dir = '/home/jason/convolutional_occupancy_networks/out/r3d_sw/'
writer = SummaryWriter(write_dir)
seed = 42
# setup_seed(seed)
lr = 0.001
BETA1 = 0.9
BETA2 = 0.999
decayRate = 0.96
epoch_num = 10000
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(BETA1, BETA2))
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)
device = 'cuda'
print_every = 50

it = 0
loss_kt = 0
iou_kt = 0
acc_kt = 0
best_iou = 0

velo_dir = '/home/jason/rellis3d/00000/velodyne'
output_dir = '/home/jason/convolutional_occupancy_networks/out/r3d_scene0/'

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)
    
if os.path.exists(write_dir):
    shutil.rmtree(write_dir)

os.mkdir(output_dir)
os.mkdir(save_dir)
os.mkdir(write_dir)

for epoch in range(epoch_num):

    model.train()
    for batch in dataloader_train:

        it += 1

        # Training
        optimizer.zero_grad()
        loss = compute_loss(batch)
        loss.backward()
        optimizer.step()
        loss_kt += loss.item()
        
        with torch.no_grad():
            # acc
            points = batch['points'].to(device)
            points_occ = batch['points.occ'].to(device)
            inputs = batch['inputs'].to(device)
            
            p_out = model(points, inputs)

            occ = (points_occ >= 0.5).cpu().numpy()
            occ_hat = (p_out.probs >= 0.5).cpu().numpy()
            acc = np.sum(occ == occ_hat, axis = 1) / occ.shape[1]

            intersection, union = iou_one_frame(occ, occ_hat, 2)
            mIou = np.mean(intersection / union)
            iou_kt += mIou

            acc_batch = np.mean(acc)
            acc_kt += acc_batch
            
            # Record
            writer.add_scalar('/Loss/Train', loss.item(), it)
            writer.add_scalar('/Accuracy/Train', acc_batch, it)
            writer.add_scalar('/mIoU/Train', mIou, it)
                
        if it % print_every == 0 :
            print("[Epoch %02d] it=%03d, loss=%.4f, acc=%.2f, iou=%.2f" % (epoch, it,loss_kt/print_every, acc_kt/print_every, iou_kt/print_every))
            loss_kt = 0
            iou_kt = 0
            acc_kt = 0
        
        
    my_lr_scheduler.step()
    
    
    # Evaluation
    model.eval()
    acc_batches = 0
    iou = 0
    with torch.no_grad():
        for bt in dataloader_eval:
            points = bt['points'].to(device)
            points_occ = bt['points.occ'].to(device)
            inputs = bt['inputs'].to(device)

            p_out = model(points, inputs)

            occ = (points_occ >= 0.5).cpu().numpy()
            occ_hat = (p_out.probs >= 0.5).cpu().numpy()
            acc = np.sum(occ == occ_hat, axis = 1) / occ.shape[1]
            acc_batch = np.mean(acc)        
            acc_batches += acc_batch
            
            intersection, union = iou_one_frame(occ, occ_hat, 2)
            mIou = np.mean(intersection/union)
            iou += mIou
            
            # Record
            writer.add_scalar('/Accuracy/Val', acc_batch, it)
            writer.add_scalar('/mIoU/Val', mIou, it)

    acc_batches /= len(dataloader_eval)
    iou /= len(dataloader_eval)
    print("acc_eval:%.2f, iou_eval:%.2f" % (acc_batches, iou))
    
    
    if iou >= best_iou:
        best_iou = iou
        torch.save(model.state_dict(), os.path.join(save_dir, "Epoch" + str(epoch) + ".pt"))
        
        # Generate output        
        
        # use frame 100 - 150:
        with torch.no_grad():

            out_dir_ep = os.path.join(output_dir, "Epoch" + str(epoch))

            if not os.path.exists(out_dir_ep):
                os.mkdir(out_dir_ep)

            for j in range(100,150):
                # points
                min_bound = [-25.6, -25.6, -2]
                max_bound = [25.6, 25.6, 1.2]
                grid_size = [256, 256, 16]
                coor_ranges = min_bound + max_bound
                voxel_sizes = [abs(coor_ranges[3] - coor_ranges[0]) / grid_size[0], 
                            abs(coor_ranges[4] - coor_ranges[1]) / grid_size[1],
                            abs(coor_ranges[5] - coor_ranges[2]) / grid_size[2]]

                x = np.linspace(min_bound[0], max_bound[0], num=int(grid_size[0])) + voxel_sizes[0] / 2
                y = np.linspace(min_bound[1], max_bound[1], num=int(grid_size[1])) + voxel_sizes[1] / 2
                z = np.linspace(min_bound[2], max_bound[2], num=int(grid_size[2])) + voxel_sizes[2] / 2
                xv, yv, zv = np.meshgrid(x, y, z, indexing="ij")

                points_t = np.stack((xv, yv, zv))
                points_t = np.moveaxis(points_t,0,-1).reshape(-1,3)
                points_t /= (2*np.max(points_t))

                pc_dir = os.path.join(velo_dir, str(j).zfill(6) + '.bin')

                # sliding windows(1/8 scale):
                scale = 8
                points_splits = np.array_split(points_t, scale)
                occ_hat_t = []
                for k in range(scale):
                    inputs_t = np.fromfile(pc_dir,dtype=np.float32).reshape(-1,4)[:,:3]  
                    
                    grid_point_mask= np.all(
                        (inputs_t < max_bound) & (inputs_t >= min_bound), axis=1)
                    inputs_t = inputs_t[grid_point_mask, :]
                    
                    # normalize
                    inputs_t /= (2*np.max(inputs_t))

                    pc_field_t = {}
                    pc_field_t[None] = inputs_t
                    pc_field_t['normals'] = np.zeros((inputs_t.shape[0],1))


                    pc_field_sub_t = SubsamplePointcloud(inputs_t.shape[0]) #10000
                    pc_field_t = pc_field_sub_t(pc_field_t)   

                    points_field_t = {}
                    points_field_t[None] = points_splits[k]
                    points_field_t['occ'] = np.zeros((points_splits[k].shape[0],1))
                    points_field_sub_t = SubsamplePoints(points_splits[k].shape[0]) #2048
                    points_field_t = points_field_sub_t(points_field_t)

                    fields_t = {}
                    data_t = {}

                    #for pc_field,points_field in carla_ds:

                    fields_t['points'] = points_field_t

                    if pc_field_t is not None:
                        fields_t['inputs'] = pc_field_t

                    for field_name_t, field_t in fields_t.items(): 

                        field_data_t = field_t
                        for k, v in field_data_t.items():
                            if k is None:
                                data_t[field_name_t] = v
                            else:
                                data_t['%s.%s' % (field_name_t, k)] = v
                    points_t = torch.tensor(data_t['points'], device=device)
                    points_t = points_t[None,:]
                    inputs_t = torch.tensor(data_t['inputs'], device=device)
                    inputs_t = inputs_t[None,:]
                    p_out_t = model(points_t, inputs_t)
                    occ_hat_t = np.append(occ_hat_t, (p_out_t.probs >= 0.5).cpu().numpy())
                occ_hat_t.astype('uint8').tofile(os.path.join(out_dir_ep, str(j).zfill(6) + '.label'))

[Epoch 00] it=050, loss=495.1362, acc=0.92, iou=0.47
[Epoch 00] it=100, loss=406.9232, acc=0.93, iou=0.52
[Epoch 00] it=150, loss=391.6013, acc=0.93, iou=0.54
[Epoch 00] it=200, loss=388.1500, acc=0.93, iou=0.55
[Epoch 00] it=250, loss=370.9835, acc=0.94, iou=0.56
[Epoch 00] it=300, loss=386.4887, acc=0.93, iou=0.55
[Epoch 00] it=350, loss=382.3016, acc=0.93, iou=0.56
[Epoch 00] it=400, loss=365.4266, acc=0.94, iou=0.57
[Epoch 00] it=450, loss=368.8153, acc=0.94, iou=0.57
[Epoch 00] it=500, loss=362.9131, acc=0.94, iou=0.58
[Epoch 00] it=550, loss=366.4593, acc=0.93, iou=0.58
[Epoch 00] it=600, loss=360.9328, acc=0.94, iou=0.58
[Epoch 00] it=650, loss=351.7759, acc=0.94, iou=0.59
[Epoch 00] it=700, loss=353.8149, acc=0.94, iou=0.58
[Epoch 00] it=750, loss=347.7660, acc=0.94, iou=0.58
[Epoch 00] it=800, loss=360.0291, acc=0.94, iou=0.58
[Epoch 00] it=850, loss=351.3802, acc=0.94, iou=0.59
[Epoch 00] it=900, loss=347.0738, acc=0.94, iou=0.59
[Epoch 00] it=950, loss=359.0088, acc=0.94, io

KeyboardInterrupt: 